In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
n = 1000
beta01, beta11 = 5,-3
beta02, beta12 = 2, 4

#set up regression mixture
x1 = np.random.uniform(0, 10, size=400)
x2 = np.random.uniform(0, 10, size=600)

y1 = beta01 + beta11*x1 + np.random.normal(scale=5.0, size=400)
y2 = beta02 + beta12*x2 + np.random.normal(scale=4.0,size=600)

X = np.concatenate([x1, x2])
Y = np.concatenate([y1, y2])


#set up 2 component mixture
a1 = np.random.normal(2, 5, size=600)
a2 = np.random.normal(5, 3, size=400)
a = np.concatenate([a1,a2])

In [3]:
np.array([1,2])[[1,0,1]]

array([2, 1, 2])

In [43]:
def e_step(y,x,params): 
    nobs, k = x.shape
    weights = []
    for param in params:
        sigma = param[-1]
        beta = np.tile(param[1:-1],nobs).reshape(nobs, k)
        mean = (beta*x).sum(axis=1)
        weights.append( stats.norm.pdf(y, loc=mean, scale=sigma) )

    #update loop variables
    weights = np.array(weights).transpose()
    denom = np.repeat(weights.sum(axis= 1),len(params)).reshape(nobs,len(params))
    weights = (weights/denom)
    return weights
        
    
def m_step(y,x,weights):
    nobs, k = x.shape
    params, se, err = [], [], 0
    
    #do lambda estimates first becaues of order
    lambs = weights.mean(axis=0)
    lamb_ses = (np.sort(weights)).std(axis=0)
    comp_order = stats.rankdata(weights.mean(axis=0),method='ordinal')
    lamb_ses = lamb_ses[comp_order-1]
    
    
    for i in comp_order:
        try:
            #component estimates
            w = weights[:,i-1]
            lamb = lambs[i-1]
            lamb_se = lamb_ses[i-1]

            #beta
            w_mat = np.diag(w)
            xx_mat = np.linalg.inv( x.transpose().dot( w_mat).dot(x) )
            beta = xx_mat.dot(x.transpose().dot(w_mat)).dot(y)

            #sigma
            mu = np.tile(beta, nobs).reshape(nobs, k)*x
            weighted_err = w*(y - mu.sum(axis=1))
            sigma =  weighted_err.std()

            #add component
            comp_param =np.concatenate(([lamb],beta,[sigma]))
            params.append(comp_param)

            #beta_se
            beta_se = np.diagonal(xx_mat*sigma**2)
            comp_se = np.concatenate(([lamb_se],beta_se))
            se.append(comp_se)

            #SSR
            err = err+weighted_err    
            
        except:        
            params.append(np.concatenate( (np.zeros(k+1),[1]) ))
            se.append( np.zeros(k+1) )
    
    return np.array(params), np.array(se), 1-err.var()/y.var()


def gen_weights(nobs,ncomp):
    weights = np.random.uniform(size=(nobs,ncomp))
    denom = np.repeat(weights.sum(axis= 1),ncomp).reshape(nobs,ncomp)
    return (weights/denom)


def estimate(y,x,ncomp):
    e = gen_weights(len(x),ncomp)
    m = None
    for i in range(15):
        m,se,r2 = m_step(y,x,e)
        e = e_step(y,x,m)
    return m, se, r2, y, x, ncomp

ests  = estimate(Y,sm.add_constant(X),2)
m, se, r2, y, x, ncomp = estimate(Y,sm.add_constant(X),2)

print m
print '----'
print se

[[ 0.41923911  5.27416605 -3.0554598   3.09273167]
 [ 0.58076089  2.98164905  3.88695089  2.90872784]]
----
[[0.13593678 0.08117191 0.00246764]
 [0.13593678 0.05639045 0.00175354]]


In [45]:
data = pd.read_csv('data/milk.csv')
reg1 = data[['WW','FMO','INC','ESC','ESTQTY','DEL','COOLER','NUMSCHL']].dropna()
reg1['QSTOP'] = reg1['ESTQTY']/(reg1['DEL']*reg1['NUMSCHL'])
reg1 = sm.add_constant(reg1)
reg1[['WW','FMO','QSTOP','DEL']] = np.log(reg1[['WW','FMO','QSTOP','DEL']])

est = estimate(np.array(reg1['WW']),np.array(reg1[['const','FMO','INC','DEL']]),2)
print est[0]

[[ 2.96667375e-01 -5.43102372e+00  1.42570827e+00  3.76729812e-02
  -9.32507833e-02  1.36412006e-02]
 [ 7.03332625e-01 -2.22353754e+00  1.53568489e-01  3.16081267e-03
   1.60746262e-02  8.75054154e-02]]


In [46]:
def write_table(fname, estimates, labels=('y',None)):
    
    #unpack relevant information
    params, se, r2, y, x, ncomp = estimates
    nobs, k = x.shape
    ylabel, xlabel = labels
    
    if xlabel == None:
        xlabel =[]
        for i in range(k):
            xlabel.append('x%s'%i)
            
    assert (k == len(xlabel)) 
    
    f = open(fname, "w+")
    
    f.write(('\\begin{center}  \n'+
            '\\begin{tabular}{lclc} \n'+
            '\\toprule \n'+
            '\\textbf{Dep. Variable:} & %s & \\textbf{  R-squared: } &  %s \\\\ \n'%(ylabel, np.round(r2,3))  ))
    
    f.write(('\\textbf{No. Observations:} & %s & & \\\\ \n'%nobs+
            '\end{tabular} \n'))
    
    
    f.write('\n\\begin{tabular}{lcccc} \n')
    for comp in range(ncomp):
        f.write('\\toprule \n')
        f.write('\\textbf{Regime %s} & \\textbf{est} & \\textbf{std err} &'%(1+comp)+ 
                '\\textbf{t} & \\textbf{P $>$ $|$ t $|$} \\\\ \n')
        f.write('\\bottomrule \\\\ \n')
        #isolate parasm
        comp_params = params[comp]
        comp_se = se[comp]
        comp_t = np.abs(comp_params[:-1]/comp_se)
        comp_p = 1 - stats.t.cdf(comp_t,df=nobs)
        
        #round everything
        comp_params = np.round(comp_params,5)
        comp_se = np.round(comp_se,5)
        comp_t = np.round(comp_t,5)
        comp_p = np.round(comp_p,5)
        
        lamb, lamb_se, lamb_t, lamb_p = comp_params[0], comp_se[0], comp_t[0], comp_p[0]
        beta, beta_se, beta_t, beta_p = comp_params[1:-1], comp_se[1:], comp_t[1:], comp_p[1:]
        sigma = comp_params[-1]
        
        
        f.write('\\textbf{lambda} & %s & (%s) & %s & %s \\\\ \\\\ \n'%(lamb,lamb_se,lamb_t,lamb_p) )
        
        for i in range(k):
            f.write('\\textbf{%s} & %s & (%s) & %s & %s \\\\ \\\\ \n'%(xlabel[i],beta[i],beta_se[i],
                                                                             beta_t[i],beta_p[i]) )
        
        f.write('\\textbf{sigma} & %s &  & & \\\\ \\\\ \n'%(sigma) )
    
    f.write('\end{tabular} \n'+
            '\end{center}\n')

write_table('out1.tex', ests)

In [47]:
write_table('prelim_results.tex', est, labels=('WW',['const','FMO','INC','COOLER']))